In [2]:
import pandas as pd
import numpy as np
import pickle

In [3]:
players_file = open(r'data/players.pkl', 'rb')
players = pickle.load(players_file)

In [4]:
results_file = open(r'data/results.pkl', 'rb')
results = pickle.load(results_file)

In [5]:
tournaments_file = open(r'data/tournaments.pkl', 'rb')
tournaments = pickle.load(tournaments_file)

In [6]:
len(tournaments), len(results), len(players)

(5528, 5528, 204063)

In [7]:
len(players)

204063

In [8]:
tournaments[2200]

{'id': 2200,
 'name': 'Бархатный сезон (КК)',
 'dateStart': '2012-09-15T00:00:00+04:00',
 'dateEnd': '2012-09-16T00:00:00+04:00',
 'type': {'id': 2, 'name': 'Обычный'},
 'season': '/seasons/44',
 'orgcommittee': [],
 'synchData': None,
 'questionQty': {'1': 15, '2': 15, '3': 15, '4': 15, '5': 15}}

In [9]:
results[2200]

[{'team': {'id': 4225, 'name': 'Соло', 'town': {'id': 97, 'name': 'Ейск'}},
  'mask': '011111100101111111111010001110111011100011100010111100101100011110011011000',
  'current': {'name': 'Соло', 'town': {'id': 97, 'name': 'Ейск'}},
  'questionsTotal': 46,
  'synchRequest': None,
  'position': 1,
  'controversials': [],
  'flags': [],
  'teamMembers': [{'flag': 'Б',
    'usedRating': 4460,
    'rating': 4460,
    'player': {'id': 29178,
     'name': 'Даниил',
     'patronymic': 'Ярославович',
     'surname': 'Синельников'}},
   {'flag': 'Б',
    'usedRating': 2010,
    'rating': 2413,
    'player': {'id': 6438,
     'name': 'Евгений',
     'patronymic': 'Александрович',
     'surname': 'Гайдуков'}},
   {'flag': 'Б',
    'usedRating': 1603,
    'rating': 2405,
    'player': {'id': 9345,
     'name': 'Алексей',
     'patronymic': 'Иванович',
     'surname': 'Диденко'}},
   {'flag': 'Б',
    'usedRating': 1169,
    'rating': 2338,
    'player': {'id': 8495,
     'name': 'Елена',
     'patr

1. Разбиваем на трейн и тест

In [11]:
tournament_type_to_keep = ['Обычный', 'Синхрон', 'Строго синхронный']

In [12]:
tournament_type = set()

cnt = 0
for key, elem in tournaments.items():
    try:
        t = elem['type']['name']
        tournament_type.add(t)
        if t in tournament_type_to_keep:
            cnt += 1
    except KeyError:
        continue
cnt / len(tournaments)

0.9129884225759769

In [13]:
train_ids = []
for key, elem in tournaments.items():
    if elem['dateStart'][:4] == '2019' and elem['type']['name'] in tournament_type_to_keep:
        train_ids.append(key)
    else:
#         print(elem['type']['name']) 
        continue
        

In [14]:
test_ids = []
for key, elem in tournaments.items():
    if elem['dateStart'][:4] == '2020' and elem['type']['name'] in tournament_type_to_keep:
        test_ids.append(key)
    else:
#         print(elem['type']['name']) 
        continue

In [15]:
train_ids = [elem['id'] for key, elem in tournaments.items() if elem['dateStart'][:4] == '2019' \
             and elem['type']['name'] in tournament_type_to_keep]
test_ids = [elem['id'] for key, elem in tournaments.items() if elem['dateStart'][:4] == '2020'
             and elem['type']['name'] in tournament_type_to_keep]

In [20]:
# player_stat = {}
# for i, ids in enumerate(train_ids):
#     if len(results[ids]) == 0 or results[ids][0] is None:
#         continue
#     task_complexity = tournaments[ids]['questionQty']  
#     for team in results[ids]:
#         try:
#             team_mask = team['mask']
#         except KeyError:
#             continue
#         if team_mask is None or len(team_mask) > 60:
# #             print(ids)
#             continue
# #         if ids == 6085:
# #             print(team_mask)
# #         print(len(team_mask), ids)
#         team_avg_rating = 0
#         for member in team['teamMembers']:
#             cur_member_id = member['player']['id']
#             if cur_member_id not in player_stat:
#                 player_stat[cur_member_id] = {}
#                 player_stat[cur_member_id]['question_result'] = []
#                 player_stat[cur_member_id]['question_result'].append(team_mask)
#                 player_stat[cur_member_id]['rating'] = []
#                 player_stat[cur_member_id]['rating'].append(member['usedRating'])
#                 player_stat[cur_member_id]['task_complexity'] = []
#                 player_stat[cur_member_id]['task_complexity'].append(task_complexity)
#                 player_stat[cur_member_id]['game'] = []
#                 player_stat[cur_member_id]['game'].append(ids)
#                 player_stat[cur_member_id]['average_team_rating'] = []
#             else:
#                 player_stat[cur_member_id]['question_result'].append(team_mask)
#                 player_stat[cur_member_id]['rating'].append(member['usedRating'])
#                 player_stat[cur_member_id]['task_complexity'].append(task_complexity)
#                 player_stat[cur_member_id]['game'].append(ids)
                
#             team_avg_rating += member['usedRating']
        
#         for member in team['teamMembers']:  # go through the same loop in order to fill avg rating
#             cur_member_id = member['player']['id']
#             player_stat[cur_member_id]['average_team_rating'].append(team_avg_rating / len(team['teamMembers']))

# for stat in player_stat:
#     assert len(player_stat[stat]['task_complexity']) == len(player_stat[stat]['rating']) == len(player_stat[stat]['question_result'])
    

In [25]:
teams_dct = {}
results_train = {}
results_test = {}

for tournament_id, teams_list in results.items():
    tournament_results = {}
    for team in teams_list:
        team_mask = team.get('mask')
        team_members = [player['player']['id'] for player in team['teamMembers']]
        
        if team_mask is None or re.findall('[^01]', team_mask) or not team_members:
            continue
          
        team_id = team['team']['id']
        team_name = team['team']['name']
        teams_dct[team_id] = team_name
        
        tournament_results[team_id] = {}
        tournament_results[team_id]['mask'] = team_mask
        tournament_results[team_id]['players'] = team_members
    
    # Уберем турниры с разным числом вопросов и разделим результаты на трейн/тест
    if len(set(list(map(len, [team['mask'] for team in tournament_results.values()])))) == 1:  
        if tournament_id in train_ids:
            results_train[tournament_id] = tournament_results
        elif tournament_id in test_ids:
            results_test[tournament_id] = tournament_results

In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder

In [30]:
ohe = OneHotEncoder(handle_unknown='ignore')

X_tr = ohe.fit_transform(train[['player_id', 'question_id']])
y_tr = train['answer']

In [31]:
lr = LogisticRegression(random_state=42, n_jobs=-1)

lr.fit(X_tr, y_tr)

/opt/homebrew/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(n_jobs=-1, random_state=42)